In [10]:
import sys
from collections import defaultdict
from itertools import *
import random
import numpy as np
import pdb
from scipy.sparse import *

from pyspark import SparkContext
sc=SparkContext.getOrCreate()

In [11]:
URM_training = sc.textFile("../train.csv")
URM_training = URM_training.map(lambda l: l.split(','))
URM_training = URM_training.filter(lambda line: 'userId' not in line)
URM_training = URM_training.map(lambda line: (int(line[0]), int(line[1]), float(line[2])))

In [17]:
URM_norm = URM_training.map(lambda x:(x[0], x[1],float(x[2])**2))

### We need to sum all data belonging to the same item
URM_norm = URM_norm.map(lambda x: (x[1], x[2])).reduceByKey(lambda x,y : x+y)

### Add a very small number to avoid division by zero
URM_map = URM_norm.map(lambda x: (x[0], x[1]+1e-6)).collectAsMap()

URM_norm = URM_training.map(lambda x:(x[0], x[1],float(x[2])/URM_map[x[1]]))

URM_norm.take(10)

[(2738, 1, 0.010309278244234246),
 (4716, 1, 0.04123711297693698),
 (13298, 1, 0.08247422595387396),
 (15122, 1, 0.04123711297693698),
 (11326, 2, 0.19999999200000032),
 (3406, 3, 0.018518518432784636),
 (3906, 3, 0.04629629608196159),
 (6345, 3, 0.027777777649176955),
 (7393, 3, 0.03703703686556927),
 (815, 4, 0.0031080031067956477)]

In [18]:
from pyspark.mllib.linalg.distributed import CoordinateMatrix, MatrixEntry

MatrixEntry_rdd = URM_training.map(lambda x: MatrixEntry(int(x[0]),int(x[1]),float(x[2])))

MatrixEntry_rdd.take(10)

[MatrixEntry(2738, 1, 1.0),
 MatrixEntry(4716, 1, 4.0),
 MatrixEntry(13298, 1, 8.0),
 MatrixEntry(15122, 1, 4.0),
 MatrixEntry(11326, 2, 5.0),
 MatrixEntry(3406, 3, 4.0),
 MatrixEntry(3906, 3, 10.0),
 MatrixEntry(6345, 3, 6.0),
 MatrixEntry(7393, 3, 8.0),
 MatrixEntry(815, 4, 8.0)]

In [19]:
from pyspark.sql import SparkSession
spark = SparkSession(sc)

matrix = CoordinateMatrix(MatrixEntry_rdd)

matrix = matrix.toBlockMatrix()

matrix

In [20]:
similarity_distributed = matrix.transpose().multiply(matrix)

print(similarity_distributed.numCols())
print(similarity_distributed.numRows())

similarity_distributed = similarity_distributed.toCoordinateMatrix()

### zero-out the diagonal
similarity_distributed = similarity_distributed.entries.map(lambda x: MatrixEntry(x.i, x.j, 0) if (x.i==x.j) else x)

similarity_display = similarity_distributed
similarity_distributed = CoordinateMatrix(similarity_distributed).toBlockMatrix()

similarity_display.take(10)

37143
37143


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 31.0 failed 1 times, most recent failure: Lost task 0.0 in stage 31.0 (TID 53, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.mllib.linalg.DenseMatrix$.zeros(Matrices.scala:461)
	at org.apache.spark.mllib.linalg.Matrix$class.multiply(Matrices.scala:105)
	at org.apache.spark.mllib.linalg.SparseMatrix.multiply(Matrices.scala:573)
	at org.apache.spark.mllib.linalg.distributed.BlockMatrix$$anonfun$23$$anonfun$apply$9$$anonfun$apply$11.apply(BlockMatrix.scala:485)
	at org.apache.spark.mllib.linalg.distributed.BlockMatrix$$anonfun$23$$anonfun$apply$9$$anonfun$apply$11.apply(BlockMatrix.scala:482)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 52149)
----------------------------------------


Traceback (most recent call last):
  File "C:\Users\Matteo\Anaconda3\envs\py35\lib\socketserver.py", line 313, in _handle_request_noblock
    self.process_request(request, client_address)
  File "C:\Users\Matteo\Anaconda3\envs\py35\lib\socketserver.py", line 341, in process_request
    self.finish_request(request, client_address)
  File "C:\Users\Matteo\Anaconda3\envs\py35\lib\socketserver.py", line 354, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "C:\Users\Matteo\Anaconda3\envs\py35\lib\socketserver.py", line 681, in __init__
    self.handle()
  File "C:/Users/Matteo/spark-2.1.0-bin-hadoop2.7\python\pyspark\accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "C:/Users/Matteo/spark-2.1.0-bin-hadoop2.7\python\pyspark\serializers.py", line 555, in read_int
    length = stream.read(4)
  File "C:\Users\Matteo\Anaconda3\envs\py35\lib\socket.py", line 576, in readinto
    return self._sock.recv_into(b)
ConnectionResetEr

In [ ]:
similarity_distributed = matrix.transpose().multiply(matrix)

print(similarity_distributed.numCols())
print(similarity_distributed.numRows())

similarity_distributed = similarity_distributed.toCoordinateMatrix()

### zero-out the diagonal
similarity_distributed = similarity_distributed.entries.map(lambda x: MatrixEntry(x.i, x.j, 0) if (x.i==x.j) else x)

similarity_display = similarity_distributed
similarity_distributed = CoordinateMatrix(similarity_distributed).toBlockMatrix()

similarity_display.take(10)

In [ ]:
itemList_rdd = sc.parallelize(itemList)


def selectUserTopK (itemId):
    
    ### We take the row we need and transform it in a dense array
    currentItemSimilarity = similarity[itemId].toarray().ravel()
    
    idx_sorted = np.argsort(currentItemSimilarity)
    
    top_k_idx = idx_sorted[-k_value:]

    values = currentItemSimilarity[top_k_idx]
    
    rows = np.arange(numItems+1)[top_k_idx]
    
    cols = np.ones(k_value, dtype=np.int32) * itemId
    
    return list(zip(rows, cols, values))

In [ ]:
userSeenItems = URM_training.map(lambda x: (x[0], [x[1]])).reduceByKey(lambda x,y : x + y)

userSeenItems_dict = userSeenItems.collectAsMap()

user_profile = CoordinateMatrix(URM_training.map(lambda x: MatrixEntry(x[0], x[1], x[2])), numRows=1, numCols=similarity_distributed.numCols())

In [ ]:
itemsScore = user_profile.toBlockMatrix().multiply(similarity_distributed)